In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
from datetime import datetime, timedelta
import secrets
import string
import re  # Import the regular expression module
import pandas as pd
import requests
import zipfile
from io import BytesIO

In [2]:
from enum import Enum
import requests
import zipfile
from io import BytesIO
import pandas as pd

class WeatherParameter(Enum):
    TU = ("air_temperature", "Temperature in °C", ['QN_9', 'eor', 'RF_TU'], '_akt')
    SD = ("sun", "Sunshine Duration", ['QN_9', 'eor', 'RF_SD'], '_akt')
    FF = ("wind", "Wind Speed in km/h", ['QN_9', 'eor', 'RF_FF'], '_akt')
    ST = ("solar", "Solar Radiation", ['QN_9', 'eor', 'RF_ST'], '_row')  # Special case for Solar Radiation
    F = ("wind_synop", "Synoptic Wind Data", ['QN_9', 'eor', 'RF_F'], '_akt')
    
    def __init__(self, category, description, columns, url_suffix):
        self.category = category
        self.description = description
        self.columns = columns
        self.url_suffix = url_suffix

class WeatherDataDownloader:
    def __init__(self, base_url=None):
        self.base_url = base_url or "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/"

    def download_and_load_weather_data(self, station_code, weather_param):
        if not isinstance(weather_param, WeatherParameter):
            raise ValueError("weather_param must be an instance of WeatherParameter Enum.")
        
        category = weather_param.category
        suffix = weather_param.url_suffix
        if weather_param == WeatherParameter.ST:
            url = f"{self.base_url}{category}/stundenwerte_{weather_param.name}_{station_code}{suffix}.zip" # Special case for Solar Radiation
        else:
            url = f"{self.base_url}{category}/recent/stundenwerte_{weather_param.name}_{station_code}{suffix}.zip" 
        response = requests.get(url)
        response.raise_for_status()
        with zipfile.ZipFile(BytesIO(response.content)) as thezip:
            # Dynamically find the correct file based on the pattern
            pattern = re.compile(f"produkt_{weather_param.name.lower()}_stunde_.*_{station_code}.txt")
            file_name = next((s for s in thezip.namelist() if pattern.match(s)), None)
            if not file_name:
                raise FileNotFoundError(f"No file matching '{pattern.pattern}' found in the ZIP archive.")
            with thezip.open(file_name) as file:
                df = pd.read_csv(file, delimiter=';', encoding='latin1')
        return df

In [55]:
# Example Usage:
downloader = WeatherDataDownloader()
station_code = '00691'
weather_param = WeatherParameter.ST # Using the Enum here
df = downloader.download_and_load_weather_data(station_code, weather_param)
if df is not None:
    print(df.head())
else:
    print("Data loading failed.")

   STATIONS_ID     MESS_DATUM  QN_592  ATMO_LBERG  FD_LBERG  FG_LBERG  \
0          691  2009010100:28       1        -999       0.0       0.0   
1          691  2009010101:28       1        -999       0.0       0.0   
2          691  2009010102:28       1        -999       0.0       0.0   
3          691  2009010103:28       1        -999       0.0       0.0   
4          691  2009010104:28       1        -999       0.0       0.0   

   SD_LBERG   ZENIT MESS_DATUM_WOZ  eor  
0         0  149.43  2009010101:00  eor  
1         0  145.44  2009010102:00  eor  
2         0  138.70  2009010103:00  eor  
3         0  130.48  2009010104:00  eor  
4         0  121.60  2009010105:00  eor  
